In [233]:
import pandas as pd
from datetime import datetime, timedelta

25.05.23 data

In [ ]:
media_agg = pd.read_parquet("./test_data/by_date_media_agg_info_temp_seller.parquet")
media_dtl = pd.read_parquet("./test_data/by_user_id_media_dtl_info_temp_seller.parquet")
user_info = pd.read_parquet("./test_data/recent_user_info_mtr_temp_seller.parquet")
time_series = pd.read_parquet("./test_data/ime_series_profile_info_temp_seller.parquet")

In [ ]:
# media_total = pd.merge(media_dtl, user_info, how='right', on='acnt_id')
# media_total = pd.read_csv("merged_temp_seller.csv", index_col=0)
# media_total.info()

25.05.26 data

In [166]:
media_agg_2 = pd.read_parquet("by_data_media_agg_info_2.parquet")
media_dtl_2 = pd.read_parquet("by_user_id_media_dtl_info_2.parquet")
user_info_2 = pd.read_parquet("recent_user_info_mtr_2.parquet")
time_series_2 = pd.read_parquet("time_series_profile_info_2.parquet")

크리에이터 활동성

- time_series_data 두개 필요
- 약 3일동안 전체 게시물이 몇 개 증가했는지 단순하게 지표 계산

In [279]:
time_series_merged_df = pd.merge(time_series, time_series_2, on='acnt_id')
time_series_merged_df

,base_ymd_x,acnt_id,follower_cnt_x,follow_cnt_x,media_cnt_x,created_dt_x,base_ymd_y,follower_cnt_y,follow_cnt_y,media_cnt_y,created_dt_y
0,2025-05-23,17841400561503844,100292,503,3690,2025-05-23 10:53:15.660,2025-05-26,100277,504,3692,2025-05-26 09:53:15.842
1,2025-05-23,17841453615191128,33003,1908,1006,2025-05-23 10:53:15.660,2025-05-26,33096,1908,1008,2025-05-26 09:53:15.842
2,2025-05-23,17841401839008777,107551,575,2658,2025-05-23 10:53:15.660,2025-05-26,107648,575,2660,2025-05-26 09:53:15.842
3,2025-05-23,17841406083664639,23505,112,368,2025-05-23 10:53:15.660,2025-05-26,23468,113,368,2025-05-26 09:53:15.842
4,2025-05-23,17841406307860250,296221,202,4507,2025-05-23 10:53:15.660,2025-05-26,296407,203,4508,2025-05-26 09:53:15.842
5,2025-05-23,17841400361359004,290328,610,2439,2025-05-23 10:53:15.660,2025-05-26,290496,609,2439,2025-05-26 09:53:15.842
6,2025-05-23,17841402936102997,43253,507,691,2025-05-23 10:53:15.660,2025-05-26,43252,506,694,2025-05-26 09:53:15.842
7,2025-05-23,17841438467094249,276215,308,1732,2025-05-23 10:53:15.660,2025-05-26,276547,308,1728,2025-05-26 09:53:15.842
8,2025-05-23,17841401506106699,263842,397,3760,2025-05-23 10:53:15.660,2025-05-26,263644,397,3763,2025-05-26 09:53:15.842
9,2025-05-23,17841400591698216,172890,320,4273,2025-05-23 10:53:15.660,2025-05-26,172858,320,4273,2025-05-26 09:53:15.842


In [280]:
time_series_merged_df['influencer_activity'] = time_series_merged_df['media_cnt_y'] - time_series_merged_df['media_cnt_x']
time_series_merged_df[['acnt_id', 'influencer_activity']]

,acnt_id,influencer_activity
0,17841400561503844,2
1,17841453615191128,2
2,17841401839008777,2
3,17841406083664639,0
4,17841406307860250,1
5,17841400361359004,0
6,17841402936102997,3
7,17841438467094249,-4
8,17841401506106699,3
9,17841400591698216,0


- 실제 게시물 업로드 날짜를 기준으로 계산할 수 있을 것 같음
- 최근 25개의 게시물의 평균 게시 간격을 계산
- 최근 25개의 게시물을 며칠에 걸쳐서 업로드 했는지에 대한 값을 계산

In [262]:
media_dtl_2_copy = media_dtl_2.copy()
media_dtl_2_copy['reg_dt'] = pd.to_datetime(media_dtl_2_copy['reg_dt'])
media_dtl_2_copy['reg_dt'] = media_dtl_2_copy['reg_dt'].dt.strftime('%Y-%m-%d')
media_dtl_2_copy = media_dtl_2_copy.sort_values(['acnt_id', 'reg_dt'])
media_dtl_2_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 275 entries, 1 to 269
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   media_id            275 non-null    object
 1   acnt_id             275 non-null    object
 2   media_type_nm       275 non-null    object
 3   reels_feed_type_nm  275 non-null    object
 4   media_url_addr      275 non-null    object
 5   media_unq_url_addr  275 non-null    object
 6   tmnl_url_addr       123 non-null    object
 7   reg_dt              275 non-null    object
 8   media_cn            273 non-null    object
 9   acnt_conn_yn        275 non-null    object
 10  feed_share_yn       0 non-null      object
 11  cmnt_actvtn_yn      0 non-null      object
dtypes: object(12)
memory usage: 27.9+ KB


In [267]:
# 게시물 간격 계산
media_dtl_2_copy['reg_dt'] = pd.to_datetime(media_dtl_2_copy['reg_dt'])
media_dtl_2_copy['prev_reg_dt'] = media_dtl_2_copy.groupby('acnt_id')['reg_dt'].shift(1)
media_dtl_2_copy['gap_days'] = (media_dtl_2_copy['reg_dt'] - media_dtl_2_copy['prev_reg_dt']).dt.days

# gap_days가 NaN인 첫 번째 포스트 제외 후 평균 간격 계산
activity_df = media_dtl_2_copy.dropna(subset=['gap_days']).groupby('acnt_id')['gap_days'].mean().reset_index()
activity_df.rename(columns={'gap_days': 'avg_upload_interval'}, inplace=True)

# 활동성 점수 계산 (간격의 역수로 환산) -> 점수 정규화 (업로드 간격이 짧을수록 점수가 높아지도록 역수를 취해서 계산한 것)
activity_df['activity_score'] = 100 / activity_df['avg_upload_interval']
activity_df

,acnt_id,avg_upload_interval,activity_score
0,17841400361359004,1.166667,85.714286
1,17841400561503844,0.500000,200.000000
2,17841400591698216,2.416667,41.379310
3,17841401473518226,3.666667,27.272727
4,17841401506106699,0.958333,104.347826
5,17841401839008777,1.041667,96.000000
6,17841402936102997,2.875000,34.782609
7,17841406083664639,2.250000,44.444444
8,17841406307860250,0.875000,114.285714
9,17841438467094249,0.708333,141.176471


In [ ]:
### 위에랑 같은 코드----

# recent_25_posts_time_table = media_dtl_2_copy.groupby(['acnt_id']).agg(
#     reg_dt_first = ('reg_dt', 'first'),
#     reg_dt_last = ('reg_dt', 'last')
# )

# recent_25_posts_time_table['reg_dt_first'] = pd.to_datetime(recent_25_posts_time_table['reg_dt_first'])
# recent_25_posts_time_table['reg_dt_last'] = pd.to_datetime(recent_25_posts_time_table['reg_dt_last'])

# recent_25_posts_time_table['period'] = recent_25_posts_time_table['reg_dt_last'] - recent_25_posts_time_table['reg_dt_first']
# recent_25_posts_time_table['period'] = recent_25_posts_time_table['period'].dt.days
# recent_25_posts_time_table['avg_upload_interval'] = recent_25_posts_time_table['period'] / (25 - 1)

# recent_25_posts_time_table

트렌드지수

- 지금 현재 일주일마다 모든 사람들이 트래킹이 되지 않고 있었기 때문에, 25년 13주차에 수집된 데이터를 기준으로 계산을 해야할 것 같음.
- 다만, 지금 없는 acnt_id가 있어서 해당 부분은 제외하고 가야할 것 같음

In [497]:
time_series_2['acnt_id'] = time_series_2['acnt_id'].astype(object)

influencer_list = time_series['acnt_id'].unique()
influencer_list
time_series_2[time_series_2['acnt_id'].isin(influencer_list)]

,base_ymd,acnt_id,follower_cnt,follow_cnt,media_cnt,created_dt
0,2025-05-26,17841400561503844,100277,504,3692,2025-05-26 09:53:15.842
1,2025-05-26,17841453615191128,33096,1908,1008,2025-05-26 09:53:15.842
2,2025-05-26,17841401839008777,107648,575,2660,2025-05-26 09:53:15.842
3,2025-05-26,17841406083664639,23468,113,368,2025-05-26 09:53:15.842
4,2025-05-26,17841406307860250,296407,203,4508,2025-05-26 09:53:15.842
5,2025-05-26,17841400361359004,290496,609,2439,2025-05-26 09:53:15.842
6,2025-05-26,17841402936102997,43252,506,694,2025-05-26 09:53:15.842
7,2025-05-26,17841438467094249,276547,308,1728,2025-05-26 09:53:15.842
8,2025-05-26,17841401506106699,263644,397,3763,2025-05-26 09:53:15.842
9,2025-05-26,17841400591698216,172858,320,4273,2025-05-26 09:53:15.842


In [498]:
time_series_merged_df = pd.merge(time_series, time_series_2, on='acnt_id')
time_series_merged_df

,base_ymd_x,acnt_id,follower_cnt_x,follow_cnt_x,media_cnt_x,created_dt_x,base_ymd_y,follower_cnt_y,follow_cnt_y,media_cnt_y,created_dt_y
0,2025-05-23,17841400561503844,100292,503,3690,2025-05-23 10:53:15.660,2025-05-26,100277,504,3692,2025-05-26 09:53:15.842
1,2025-05-23,17841453615191128,33003,1908,1006,2025-05-23 10:53:15.660,2025-05-26,33096,1908,1008,2025-05-26 09:53:15.842
2,2025-05-23,17841401839008777,107551,575,2658,2025-05-23 10:53:15.660,2025-05-26,107648,575,2660,2025-05-26 09:53:15.842
3,2025-05-23,17841406083664639,23505,112,368,2025-05-23 10:53:15.660,2025-05-26,23468,113,368,2025-05-26 09:53:15.842
4,2025-05-23,17841406307860250,296221,202,4507,2025-05-23 10:53:15.660,2025-05-26,296407,203,4508,2025-05-26 09:53:15.842
5,2025-05-23,17841400361359004,290328,610,2439,2025-05-23 10:53:15.660,2025-05-26,290496,609,2439,2025-05-26 09:53:15.842
6,2025-05-23,17841402936102997,43253,507,691,2025-05-23 10:53:15.660,2025-05-26,43252,506,694,2025-05-26 09:53:15.842
7,2025-05-23,17841438467094249,276215,308,1732,2025-05-23 10:53:15.660,2025-05-26,276547,308,1728,2025-05-26 09:53:15.842
8,2025-05-23,17841401506106699,263842,397,3760,2025-05-23 10:53:15.660,2025-05-26,263644,397,3763,2025-05-26 09:53:15.842
9,2025-05-23,17841400591698216,172890,320,4273,2025-05-23 10:53:15.660,2025-05-26,172858,320,4273,2025-05-26 09:53:15.842


In [499]:
time_series_merged_df['trend_score'] = ((time_series_merged_df['follower_cnt_y'] - time_series_merged_df['follower_cnt_x']) / (time_series_merged_df['follower_cnt_x'])) * 100
time_series_merged_df[['acnt_id', 'trend_score']]

,acnt_id,trend_score
0,17841400561503844,-0.014956
1,17841453615191128,0.281793
2,17841401839008777,0.090190
3,17841406083664639,-0.157413
4,17841406307860250,0.062791
5,17841400361359004,0.057866
6,17841402936102997,-0.002312
7,17841438467094249,0.120196
8,17841401506106699,-0.075045
9,17841400591698216,-0.018509


##### 팔로워 참여도

- 26일 데이터를 기준으로 계산 진행
- 단일 프소트 기준이 아닌 전체 포스트 평균을 측정
- media_dtl & media_agg & user_data를 병합한 데이터

---------------------------------------------------------

- 좋아요 비공개인 경우에는 각각의 중앙값으로 대체 -> 문제는 아예 없는 사람은 어떻게 해야하는건지...
- estimated_total_engagement = avg_engagement_per_post * total_post_count
- avg_engagement_per_post = like+comments / 25
- total_post_count -> user_data

In [167]:
media_engagement_merged_df = pd.merge(media_dtl, media_agg, on='media_id')
media_engagement_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   media_id            275 non-null    object        
 1   acnt_id             275 non-null    object        
 2   media_type_nm       275 non-null    object        
 3   reels_feed_type_nm  275 non-null    object        
 4   media_url_addr      275 non-null    object        
 5   media_unq_url_addr  275 non-null    object        
 6   tmnl_url_addr       124 non-null    object        
 7   reg_dt              275 non-null    object        
 8   media_cn            273 non-null    object        
 9   acnt_conn_yn        275 non-null    object        
 10  feed_share_yn       0 non-null      object        
 11  cmnt_actvtn_yn      0 non-null      object        
 12  base_ymd            275 non-null    object        
 13  cmnt_cnt            275 non-null    int64         

In [168]:
influencer_list

array(['17841400561503844', '17841453615191128', '17841401839008777',
       '17841406083664639', '17841406307860250', '17841400361359004',
       '17841402936102997', '17841438467094249', '17841401506106699',
       '17841400591698216', '17841401473518226'], dtype=object)

In [169]:
media_engagement_merged_df.groupby(['acnt_id'])['like_cnt'].median()

acnt_id
17841400361359004      NaN
17841400561503844    137.0
17841400591698216    810.0
17841401473518226      NaN
17841401506106699    791.0
17841401839008777    479.5
17841402936102997     96.0
17841406083664639     83.0
17841406307860250    684.0
17841438467094249    293.0
17841453615191128    348.0
Name: like_cnt, dtype: float64

In [170]:
fillna_dict = {
    '17841400591698216': 810,
    '17841401506106699': 791,
    '17841401839008777': 480,
    '17841402936102997': 96,
    '17841406083664639': 83,
    '17841400361359004' : 0,
    '17841401473518226' : 0
}

for acnt_id, default_like in fillna_dict.items():
    condition = media_engagement_merged_df['acnt_id'] == acnt_id
    media_engagement_merged_df.loc[condition, 'like_cnt'] = (
        media_engagement_merged_df.loc[condition, 'like_cnt'].fillna(default_like)
    )

In [ ]:
media_engagement_merged_df = pd.merge(media_engagement_merged_df, user_info_2,  on='acnt_id', how = 'left')

In [178]:
media_engagement_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   media_id                275 non-null    object        
 1   acnt_id                 275 non-null    object        
 2   media_type_nm           275 non-null    object        
 3   reels_feed_type_nm      275 non-null    object        
 4   media_url_addr          275 non-null    object        
 5   media_unq_url_addr      275 non-null    object        
 6   tmnl_url_addr           124 non-null    object        
 7   reg_dt                  275 non-null    object        
 8   media_cn                273 non-null    object        
 9   acnt_conn_yn_x          275 non-null    object        
 10  feed_share_yn           0 non-null      object        
 11  cmnt_actvtn_yn          0 non-null      object        
 12  base_ymd                275 non-null    object    

In [182]:
media_engagement_merged_df_2 = media_engagement_merged_df[['acnt_id', 'media_id', 'follower_cnt', 'follow_cnt', 'like_cnt', 'cmnt_cnt', 'media_cnt']]

In [402]:
media_engagement_merged_df_2

,acnt_id,media_id,follower_cnt,follow_cnt,like_cnt,cmnt_cnt,media_cnt
0,17841400361359004,17904456294145849,290496,609,0.0,19,2439
1,17841400361359004,17943099914871657,290496,609,0.0,36,2439
2,17841400361359004,17946018998972898,290496,609,0.0,24,2439
3,17841400361359004,17977890356839786,290496,609,0.0,0,2439
4,17841400361359004,18013855184713804,290496,609,0.0,224,2439
...,...,...,...,...,...,...,...
270,17841453615191128,18081388285735431,33096,1908,345.0,67,1008
271,17841453615191128,18091990390581574,33096,1908,879.0,155,1008
272,17841453615191128,18092733172572211,33096,1908,532.0,88,1008
273,17841453615191128,18298515442172742,33096,1908,306.0,93,1008


In [433]:
engaged_df = media_engagement_merged_df_2.groupby(['acnt_id']).agg({
    'like_cnt' : 'sum',
    'cmnt_cnt' : 'sum',
    'media_cnt': 'first',
    'follower_cnt' : 'first',
    'follow_cnt' : 'first'
}).reset_index()

engaged_df['avg_engagement_per_post'] = ((engaged_df['like_cnt'] + engaged_df['cmnt_cnt']) / 25)
engaged_df['estimated_total_engagement'] = engaged_df['avg_engagement_per_post'] * engaged_df['media_cnt']
engaged_df['follower_total_engagement'] = (engaged_df['estimated_total_engagement'] / engaged_df['follower_cnt']) * 100
# engaged_df['checking'] = engaged_df['avg_engagement_per_post'] / engaged_df['follower_cnt']
engaged_df

,acnt_id,like_cnt,cmnt_cnt,media_cnt,follower_cnt,follow_cnt,avg_engagement_per_post,estimated_total_engagement,follower_total_engagement
0,17841400361359004,0.0,3281,2439,290496,609,131.24,320094.36,110.188904
1,17841400561503844,4067.0,188,3692,100277,504,170.20,628378.40,626.642600
2,17841400591698216,20187.0,703,4273,172858,320,835.60,3570518.80,2065.579146
3,17841401473518226,0.0,316,1438,37905,1063,12.64,18176.32,47.952302
4,17841401506106699,35075.0,1620,3763,263644,397,1467.80,5523331.40,2094.996055
5,17841401839008777,11999.0,4087,2660,107648,575,643.44,1711550.40,1589.950951
6,17841402936102997,2495.0,104,694,43252,506,103.96,72148.24,166.809026
7,17841406083664639,5544.0,4978,368,23468,113,420.88,154883.84,659.978865
8,17841406307860250,33869.0,794,4508,296407,203,1386.52,6250432.16,2108.732979
9,17841438467094249,9317.0,5052,1728,276547,308,574.76,993185.28,359.137969


##### 팔로워 충성도

- user_info or time_series 데이터가 필요
- 트렌드 지수는 time_series 데이터를 가지고 단순히 빼기를 진행했었음

In [339]:
time_series_merged_df = pd.merge(time_series, time_series_2, on='acnt_id')
time_series_merged_df_copy = time_series_merged_df[['acnt_id', 'follower_cnt_x', 'follower_cnt_y']]

In [340]:
time_series_merged_df_copy['follower_change'] = (time_series_merged_df_copy['follower_cnt_y'] - time_series_merged_df_copy['follower_cnt_x'])

C:\Users\ehddl\AppData\Local\Temp\ipykernel_1732\1470226196.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_series_merged_df_copy['follower_change'] = (time_series_merged_df_copy['follower_cnt_y'] - time_series_merged_df_copy['follower_cnt_x'])


In [341]:
time_series_merged_df_copy

,acnt_id,follower_cnt_x,follower_cnt_y,follower_change
0,17841400561503844,100292,100277,-15
1,17841453615191128,33003,33096,93
2,17841401839008777,107551,107648,97
3,17841406083664639,23505,23468,-37
4,17841406307860250,296221,296407,186
5,17841400361359004,290328,290496,168
6,17841402936102997,43253,43252,-1
7,17841438467094249,276215,276547,332
8,17841401506106699,263842,263644,-198
9,17841400591698216,172890,172858,-32


In [342]:
def estimate_new_follower(row):
    if row['follower_change'] < 0:
        return 0
    else:
        return row['follower_change']

time_series_merged_df_copy['new_follower'] = time_series_merged_df_copy.apply(estimate_new_follower, axis=1)

# 충성도 계산
# time_series_merged_df_copy = time_series_merged_time_series_merged_df_copy_copy.copy()
time_series_merged_df_copy['unfollowed'] = time_series_merged_df_copy['follower_cnt_x'] + time_series_merged_df_copy['new_follower'] - time_series_merged_df_copy['follower_cnt_y']
time_series_merged_df_copy['follower_retention_rate'] = ((time_series_merged_df_copy['follower_cnt_x'] - time_series_merged_df_copy['unfollowed']) / time_series_merged_df_copy['follower_cnt_x']) * 100
time_series_merged_df_copy['follower_retention_rate'] = time_series_merged_df_copy['follower_retention_rate'].round(2)

time_series_merged_df_copy

C:\Users\ehddl\AppData\Local\Temp\ipykernel_1732\4021123160.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_series_merged_df_copy['new_follower'] = time_series_merged_df_copy.apply(estimate_new_follower, axis=1)
C:\Users\ehddl\AppData\Local\Temp\ipykernel_1732\4021123160.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_series_merged_df_copy['unfollowed'] = time_series_merged_df_copy['follower_cnt_x'] + time_series_merged_df_copy['new_follower'] - time_series_merged_df_copy['follower_cnt_y'

,acnt_id,follower_cnt_x,follower_cnt_y,follower_change,new_follower,unfollowed,follower_retention_rate
0,17841400561503844,100292,100277,-15,0,15,99.99
1,17841453615191128,33003,33096,93,93,0,100.00
2,17841401839008777,107551,107648,97,97,0,100.00
3,17841406083664639,23505,23468,-37,0,37,99.84
4,17841406307860250,296221,296407,186,186,0,100.00
5,17841400361359004,290328,290496,168,168,0,100.00
6,17841402936102997,43253,43252,-1,0,1,100.00
7,17841438467094249,276215,276547,332,332,0,100.00
8,17841401506106699,263842,263644,-198,0,198,99.92
9,17841400591698216,172890,172858,-32,0,32,99.98


In [ ]:
# def estimate_loyalty(row):
#     if row['follower_change'] < 0:
#         return ((row['follower_cnt_x'] + row['follower_change']) / row['follower_cnt_x']) * 100
#     else:
#         return 100.0

# time_series_merged_df_copy['follower_loyalty'] = time_series_merged_df_copy.apply(estimate_loyalty, axis=1)
# time_series_merged_df_copy['follower_loyalty'] = time_series_merged_df_copy['follower_loyalty'].round(2)


콘텐츠 효율성 평가

- 콘텐츠 1개당 팔로워 대비 반응량(like+comments)
- media_agg & media_dtl & acnt_id가 필요 -> 전부 병합한 게 media_engagement_merged_df

In [343]:
media_engagement_merged_df_2

,acnt_id,media_id,follower_cnt,follow_cnt,like_cnt,cmnt_cnt,media_cnt
0,17841400361359004,17904456294145849,290496,609,0.0,19,2439
1,17841400361359004,17943099914871657,290496,609,0.0,36,2439
2,17841400361359004,17946018998972898,290496,609,0.0,24,2439
3,17841400361359004,17977890356839786,290496,609,0.0,0,2439
4,17841400361359004,18013855184713804,290496,609,0.0,224,2439
...,...,...,...,...,...,...,...
270,17841453615191128,18081388285735431,33096,1908,345.0,67,1008
271,17841453615191128,18091990390581574,33096,1908,879.0,155,1008
272,17841453615191128,18092733172572211,33096,1908,532.0,88,1008
273,17841453615191128,18298515442172742,33096,1908,306.0,93,1008


In [353]:
media_engagement_merged_df_3 = media_engagement_merged_df_2.copy()

media_engagement_merged_df_3['post_efficiency'] = ((media_engagement_merged_df_3['like_cnt'] + media_engagement_merged_df_3['cmnt_cnt']) / media_engagement_merged_df_3['follower_cnt']) * 100
media_engagement_merged_df_3

,acnt_id,media_id,follower_cnt,follow_cnt,like_cnt,cmnt_cnt,media_cnt,post_efficiency
0,17841400361359004,17904456294145849,290496,609,0.0,19,2439,0.006541
1,17841400361359004,17943099914871657,290496,609,0.0,36,2439,0.012393
2,17841400361359004,17946018998972898,290496,609,0.0,24,2439,0.008262
3,17841400361359004,17977890356839786,290496,609,0.0,0,2439,0.000000
4,17841400361359004,18013855184713804,290496,609,0.0,224,2439,0.077109
...,...,...,...,...,...,...,...,...
270,17841453615191128,18081388285735431,33096,1908,345.0,67,1008,1.244863
271,17841453615191128,18091990390581574,33096,1908,879.0,155,1008,3.124245
272,17841453615191128,18092733172572211,33096,1908,532.0,88,1008,1.873338
273,17841453615191128,18298515442172742,33096,1908,306.0,93,1008,1.205584


In [350]:
post_efficiency_df = media_engagement_merged_df_3.groupby('acnt_id')['post_efficiency'].mean().reset_index()
post_efficiency_df.rename(columns={'post_efficiency': 'avg_post_efficiency'}, inplace=True)


In [351]:
post_efficiency_df

,acnt_id,avg_post_efficiency
0,17841400361359004,0.045178
1,17841400561503844,0.169730
2,17841400591698216,0.483403
3,17841401473518226,0.033347
4,17841401506106699,0.556736
5,17841401839008777,0.597726
6,17841402936102997,0.240359
7,17841406083664639,1.793421
8,17841406307860250,0.467776
9,17841438467094249,0.207834


##### 크리에이터 광고 효율성

In [436]:
media_engagement_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   media_id                275 non-null    object        
 1   acnt_id                 275 non-null    object        
 2   media_type_nm           275 non-null    object        
 3   reels_feed_type_nm      275 non-null    object        
 4   media_url_addr          275 non-null    object        
 5   media_unq_url_addr      275 non-null    object        
 6   tmnl_url_addr           124 non-null    object        
 7   reg_dt                  275 non-null    object        
 8   media_cn                273 non-null    object        
 9   acnt_conn_yn_x          275 non-null    object        
 10  feed_share_yn           0 non-null      object        
 11  cmnt_actvtn_yn          0 non-null      object        
 12  base_ymd                275 non-null    object    

In [590]:
revenue_dic = {
    'acnt_nm' : ['s_h_j_', 'siwolbubu_hyun', 'bong_camper83', 'binwoos', 'seojinii_', 'tingkerhee'],
    'sell_type' : ['flexmatch', 'other', 'flexmatch', 'flexmatch', 'other', 'other'],
    'total_revenue' : [6906000, 10937105, 7233100, 8759000, 7939664, 38449720],
    'total_order_cnt' : [84, 132, 122, 88, 89, 471]
    }

revenue_df = pd.DataFrame(revenue_dic)
revenue_df

,acnt_nm,sell_type,total_revenue,total_order_cnt
0,s_h_j_,flexmatch,6906000,84
1,siwolbubu_hyun,other,10937105,132
2,bong_camper83,flexmatch,7233100,122
3,binwoos,flexmatch,8759000,88
4,seojinii_,other,7939664,89
5,tingkerhee,other,38449720,471


In [591]:
revenue_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   acnt_nm          6 non-null      object
 1   sell_type        6 non-null      object
 2   total_revenue    6 non-null      int64 
 3   total_order_cnt  6 non-null      int64 
dtypes: int64(2), object(2)
memory usage: 320.0+ bytes


In [592]:
revenue_merged_df = pd.merge(media_engagement_merged_df, revenue_df, on='acnt_nm', how='left')
revenue_merged_df = revenue_merged_df[['acnt_id', 'acnt_nm', 'follower_cnt', 'follow_cnt', 'media_cnt', 'sell_type', 'total_revenue', 'total_order_cnt', 'media_id', 'like_cnt', 'cmnt_cnt']]
revenue_merged_df

,acnt_id,acnt_nm,follower_cnt,follow_cnt,media_cnt,sell_type,total_revenue,total_order_cnt,media_id,like_cnt,cmnt_cnt
0,17841400361359004,s_h_j_,290496,609,2439,flexmatch,6906000.0,84.0,17904456294145849,0.0,19
1,17841400361359004,s_h_j_,290496,609,2439,flexmatch,6906000.0,84.0,17943099914871657,0.0,36
2,17841400361359004,s_h_j_,290496,609,2439,flexmatch,6906000.0,84.0,17946018998972898,0.0,24
3,17841400361359004,s_h_j_,290496,609,2439,flexmatch,6906000.0,84.0,17977890356839786,0.0,0
4,17841400361359004,s_h_j_,290496,609,2439,flexmatch,6906000.0,84.0,18013855184713804,0.0,224
...,...,...,...,...,...,...,...,...,...,...,...
270,17841453615191128,bong_camper83,33096,1908,1008,flexmatch,7233100.0,122.0,18081388285735431,345.0,67
271,17841453615191128,bong_camper83,33096,1908,1008,flexmatch,7233100.0,122.0,18091990390581574,879.0,155
272,17841453615191128,bong_camper83,33096,1908,1008,flexmatch,7233100.0,122.0,18092733172572211,532.0,88
273,17841453615191128,bong_camper83,33096,1908,1008,flexmatch,7233100.0,122.0,18298515442172742,306.0,93


In [593]:
revenue_merged_df['post_efficiency'] = ((revenue_merged_df['like_cnt'] + revenue_merged_df['cmnt_cnt']) / revenue_merged_df['follower_cnt']) * 100
revenue_df_total = revenue_merged_df.groupby(['acnt_id', 'acnt_nm']).agg({
    'post_efficiency' : 'mean',
    'total_order_cnt' : 'first'
}).dropna()

revenue_df_total

,,post_efficiency,total_order_cnt
acnt_id,acnt_nm,,
17841400361359004,s_h_j_,0.045178,84.0
17841400561503844,binwoos,0.169730,88.0
17841400591698216,tingkerhee,0.483403,471.0
17841401506106699,siwolbubu_hyun,0.556736,132.0
17841402936102997,seojinii_,0.240359,89.0
17841453615191128,bong_camper83,2.506526,122.0


In [594]:
revenue_df_total['advertisement_efficiency'] = (revenue_df_total['total_order_cnt'] / revenue_df_total['post_efficiency']) 
revenue_df_total.sort_values(by='advertisement_efficiency', ascending=False)

,,post_efficiency,total_order_cnt,advertisement_efficiency
acnt_id,acnt_nm,,,
17841400361359004,s_h_j_,0.045178,84.0,1859.316062
17841400591698216,tingkerhee,0.483403,471.0,974.343202
17841400561503844,binwoos,0.169730,88.0,518.470975
17841402936102997,seojinii_,0.240359,89.0,370.279723
17841401506106699,siwolbubu_hyun,0.556736,132.0,237.096389
17841453615191128,bong_camper83,2.506526,122.0,48.672935


팔로워를 생각하지 않고, 25개의 콘텐츠 단위당 반응 비율을 계산

In [486]:
revenue_merged_df['engagement_per_post'] = ((revenue_merged_df['like_cnt'] + revenue_merged_df['cmnt_cnt']) / 25)
revenue_df_total = revenue_merged_df.groupby(['acnt_id', 'acnt_nm']).agg({
    'engagement_per_post' : 'mean',
    'total_revenue' : 'first',
    'total_order_cnt' : 'first'
}).dropna()

# revenue_df_total.sort_values(by='total_order_cnt', ascending=False)

In [489]:
revenue_df_total['advertisement_efficiency'] = (revenue_df_total['total_order_cnt'] / revenue_df_total['engagement_per_post'] * 25) * 100
revenue_df_total.sort_values(by='advertisement_efficiency', ascending=False)

,,engagement_per_post,total_revenue,total_order_cnt,advertisement_efficiency
acnt_id,acnt_nm,,,,
17841402936102997,seojinii_,4.1584,7939664.0,89.0,53506.156214
17841400361359004,s_h_j_,5.2496,6906000.0,84.0,40003.047851
17841400591698216,tingkerhee,33.4240,38449720.0,471.0,35229.176640
17841400561503844,binwoos,6.8080,8759000.0,88.0,32314.923619
17841453615191128,bong_camper83,33.1824,7233100.0,122.0,9191.619654
17841401506106699,siwolbubu_hyun,58.7120,10937105.0,132.0,5620.656765


콘텐츠 효율성이 아닌 팔로워 참여도를 기준으로 계산했을 때

In [469]:
engaged_df_2 = engaged_df.copy()
# engaged_df_2

In [470]:
revenue_merged_df_2 = revenue_merged_df.groupby(['acnt_id', 'acnt_nm'])[['total_revenue', 'total_order_cnt']].first().dropna().reset_index()
# revenue_merged_df_2

In [471]:
revenue_df_total_2 = pd.merge(engaged_df_2, revenue_merged_df_2, on='acnt_id')
# revenue_df_total_2

In [472]:
revenue_df_total_2['advertisement_efficiency'] = revenue_df_total_2['total_order_cnt'] / revenue_df_total_2['follower_total_engagement']
revenue_df_total_2[['acnt_id', 'acnt_nm', 'total_revenue', 'total_order_cnt', 'follower_total_engagement', 'advertisement_efficiency']].sort_values(by='advertisement_efficiency', ascending=False)

,acnt_id,acnt_nm,total_revenue,total_order_cnt,follower_total_engagement,advertisement_efficiency
0,17841400361359004,s_h_j_,6906000.0,84.0,110.188904,0.762327
4,17841402936102997,seojinii_,7939664.0,89.0,166.809026,0.533544
2,17841400591698216,tingkerhee,38449720.0,471.0,2065.579146,0.228023
1,17841400561503844,binwoos,8759000.0,88.0,626.642600,0.140431
3,17841401506106699,siwolbubu_hyun,10937105.0,132.0,2094.996055,0.063007
5,17841453615191128,bong_camper83,7233100.0,122.0,2526.578680,0.048287


table merged

In [583]:
# 크리에이터 활동성
creator_activity_score = activity_df[['acnt_id', 'avg_upload_interval']]

In [584]:
# 트렌드지수
creator_trend_score = time_series_merged_df[['acnt_id', 'trend_score']]

In [585]:
# 팔로워 참여도
follower_engagement = engaged_df[['acnt_id', 'follower_total_engagement']]

In [586]:
# 팔로워 충성도
follower_loyalty = time_series_merged_df_copy[['acnt_id', 'follower_retention_rate']]

In [587]:
# 콘텐츠 효율성
post_efficiency = post_efficiency_df.copy()

In [595]:
# 광고효율성
revenue_df_total = revenue_df_total.reset_index()
advertisement_efficiency = revenue_df_total[['acnt_id', 'advertisement_efficiency']]

In [596]:
df_list = [creator_activity_score, creator_trend_score, follower_engagement, follower_loyalty, post_efficiency, advertisement_efficiency]

In [597]:
from functools import reduce

flexmatch_score = reduce(lambda left, right: pd.merge(left, right, on='acnt_id', how='left'), df_list)

In [598]:
user_info_nm = user_info_2[['acnt_id', 'acnt_nm']]
flexmatch_score = pd.merge(flexmatch_score, user_info_nm, on='acnt_id')

In [604]:
flexmatch_score = flexmatch_score[['acnt_id', 'acnt_nm', 'avg_upload_interval', 'trend_score', 'follower_total_engagement', 'follower_retention_rate', 'avg_post_efficiency', 'advertisement_efficiency']]
flexmatch_score.rename(columns={
    'avg_upload_interval' : '크리에이터 활동성',
    'trend_score' : '트렌드지수(팔로워순증가량)', 
    'follower_total_engagement' : '콘텐츠 참여도' ,
    'follower_retention_rate' : '팔로워 충성도',
    'avg_post_efficiency' : '콘텐츠 효율성',
    'advertisement_efficiency' : '광고 효율성'
}, inplace=True)

In [605]:
flexmatch_score_2 = flexmatch_score.copy()
flexmatch_score_2.dropna(inplace=True)

In [606]:
flexmatch_score_2

,acnt_id,acnt_nm,크리에이터 활동성,트렌드지수(팔로워순증가량),콘텐츠 참여도,팔로워 충성도,콘텐츠 효율성,광고 효율성
0,17841400361359004,s_h_j_,1.166667,0.057866,110.188904,100.00,0.045178,1859.316062
1,17841400561503844,binwoos,0.500000,-0.014956,626.642600,99.99,0.169730,518.470975
2,17841400591698216,tingkerhee,2.416667,-0.018509,2065.579146,99.98,0.483403,974.343202
4,17841401506106699,siwolbubu_hyun,0.958333,-0.075045,2094.996055,99.92,0.556736,237.096389
6,17841402936102997,seojinii_,2.875000,-0.002312,166.809026,100.00,0.240359,370.279723
10,17841453615191128,bong_camper83,1.500000,0.281793,2526.578680,100.00,2.506526,48.672935


In [608]:
# from sklearn.preprocessing import MinMaxScaler

# columns = flexmatch_score_2.select_dtypes(include='float64').columns
# minmax = MinMaxScaler(feature_range=(0, 5))
# flexmatch_score_norm = minmax.fit_transform(flexmatch_score_2[columns])

# flexmatch_score_norm_df = pd.DataFrame(flexmatch_score_norm, columns=columns, index=flexmatch_score_2.index)
# flexmatch_score_norm_df['acnt_id'] = flexmatch_score_2['acnt_id']
# flexmatch_score_norm_df['acnt_nm'] = flexmatch_score_2['acnt_nm']

# flexmatch_score_norm_df['avg_upload_interval'] = 5 - flexmatch_score_norm_df['avg_upload_interval']
# flexmatch_score_norm_df


In [ ]:
from sklearn.preprocessing import MinMaxScaler

columns = flexmatch_score_2.select_dtypes(include='float64').columns
minmax = MinMaxScaler(feature_range=(0, 5))
flexmatch_score_norm = minmax.fit_transform(flexmatch_score_2[columns])

flexmatch_score_norm_df = pd.DataFrame(flexmatch_score_norm, columns=columns, index=flexmatch_score_2.index)
flexmatch_score_norm_df['acnt_id'] = flexmatch_score_2['acnt_id']
flexmatch_score_norm_df['acnt_nm'] = flexmatch_score_2['acnt_nm']

flexmatch_score_norm_df['크리에이터 활동성'] = 5 - flexmatch_score_norm_df['크리에이터 활동성']
flexmatch_score_norm_df

# 지금 이거 정규화 할 때는 그냥 이사람들을 전부 다 합쳐서 함께 정규화를 진행해서 그런데, 원래라면은 팔로워를 기준으로 뭐 메가 인플루언서, 마이크로 인플루언서 이런식으로 기준을 나눠서 정규화를 해야 조금 더 정확한 값이 될 것 같음.


,크리에이터 활동성,트렌드지수(팔로워순증가량),콘텐츠 참여도,팔로워 충성도,콘텐츠 효율성,광고 효율성,acnt_id,acnt_nm
0,3.596491,1.862339,0.000000,5.000,0.000000,5.000000,17841400361359004,s_h_j_
1,5.000000,0.841960,1.068647,4.375,0.253016,1.297324,17841400561503844,binwoos
2,0.964912,0.792182,4.046099,3.750,0.890213,2.556192,17841400591698216,tingkerhee
4,4.035088,0.000000,4.106968,0.000,1.039182,0.520322,17841401506106699,siwolbubu_hyun
6,0.000000,1.019133,0.117159,5.000,0.396492,0.888101,17841402936102997,seojinii_
10,2.894737,5.000000,5.000000,5.000,5.000000,0.000000,17841453615191128,bong_camper83


In [ ]:
# flexmatch_score_norm_df.to_csv("flexmatch_score_test.csv")